In [24]:
import time
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

#df = pd.read_csv(city_list)

def airdna_scraper(cities) :
    
    opts = FirefoxOptions()
    # opts.add_argument("--headless")

    driver = webdriver.Firefox(options=opts)
    driver.get("https://app.airdna.co/data/login")
    
    WebDriverWait(driver, 60).until(
        EC.visibility_of_all_elements_located((By.ID, "login-email"))
    )
    
    print('Logging in . . .')
    
    login = driver.find_element(By.ID, "login-email")  
    login.send_keys('castillosam27@gmail.com')
    
    password = driver.find_element(By.ID, 'login-password')
    password.send_keys("D@ncIngtOfU1")
    
    time.sleep(1)
    
    password.send_keys(Keys.ENTER)

    time.sleep(6)

    
    listing_data = []

    print('Searching for listings . . .')

    listings_captured = 0
    max_listings = 20


    while listings_captured < max_listings:

        listings = driver.find_elements(By.TAG_NAME, "a")
        
        for listing in listings:
            if listings_captured >= max_listings:
                break
            
            driver.execute_script("arguments[0].scrollIntoView();", listing)
        
            try:

                name_score = listing.find_elements(By.TAG_NAME, 'h6')
                name = name_score[0].text
                print(name)
            
                score = name_score[1].text
                print(score)
                
                info = listing.find_elements(By.TAG_NAME, 'p')
                for i in info:
                    print(i.text)
    
                if name in cities:

                # Collecting the data
                    listing_data.append({
                        "name": name,
                        "score": score,
                        "info": [i.text for i in info]
                    })

                    listings_captured_captured += 1

            except Exception as e:
                print(f'Could not extract listing data: {e}')
                continue

        # Scroll down to load more reviews if needed
        if listings_captured < max_listings:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, 20).until(
                EC.visibility_of_all_elements_located((By.TAG_NAME, 'a')))

    driver.quit()

    return listing_data

In [2]:
city_list = ['San Juan','Montgomery','Kauai','Jackson','New York','Birmingham','Knoxville','Montgomery','New Haven','San Diego','Helena','St. Louis','Chattanooga','Louisville','Lexington']

In [4]:
info = airdna_scraper(city_list)

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElements@chrome://remote/content/marionette/driver.sys.mjs:1595:15
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [4]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#df = pd.read_csv(city_list)

def login_to_airdna(driver):
    driver.get("https://app.airdna.co/data/login")
    
    WebDriverWait(driver, 60).until(
        EC.visibility_of_all_elements_located((By.ID, "login-email"))
    )
    
    WebDriverWait(driver, 60).until(
        EC.visibility_of_all_elements_located((By.ID, "login-password"))
    )

    print('Logging in . . .')

    login = driver.find_element(By.ID, "login-email")
    login.send_keys('castillosam27@gmail.com')

    password = driver.find_element(By.ID, 'login-password')
    password.send_keys("D@ncIngtOfU1")

    time.sleep(1)

    password.send_keys(Keys.ENTER)
    time.sleep(10)  # Wait for the page to load after login

def scrape_listings(driver, cities):
    listing_data = []
    listings_captured = 0

    print('Searching for listings . . .')
    
    while 2:
        
        try:
            listings = driver.find_elements(By.TAG_NAME, "a")
            print(f"Detected {len(listings)} listings . . .")
            
        except Exception as e:
            print(f'Could not detect listings: {e}')
            continue
        
        listings_seen = []
        
        for listing in listings:
                        
            try:
                
                driver.execute_script("arguments[0].scrollIntoView();", listing)

                name_score = listing.find_elements(By.TAG_NAME, 'h6')
                if len(name_score) < 2:
                    continue  # Skip if the necessary elements are not found

                name = name_score[1].text
                score = name_score[0].text

                info = listing.find_elements(By.TAG_NAME, 'p')

                print(f'Name: {name}')
                print(f'Score: {score}')

                for i in info:
                    print(f'Info: {i.text}')

                if name in cities:
                    # Collecting the data
                    listing_data.append({
                        "name": name,
                        "score": score,
                        "info": [i.text for i in info]
                    })

                listings_captured += 1
                listings_seen.append(listing)
                
                print(f"Listings Captured: {listings_captured}/{max_listings}")
                print()

            except Exception as e:
                print(f'Could not extract listing data: {e}')
                continue

        # Scroll down to load more reviews if needed
            try:
                #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                driver.execute_script("arguments[0].scrollIntoView();", listings[-1])

                time.sleep(10)  # Allow time for the page to load more listings
                
                listings = driver.find_elements(By.TAG_NAME, "a")[listings_captured:]
                
            except Exception as e:
                print(f'Could not scroll the page: {e}')
                continue

    return listing_data

def airdna_scraper(cities):
    opts = FirefoxOptions()
    # opts.add_argument("--headless")
    driver = webdriver.Firefox(options=opts)

    try:
        login_to_airdna(driver)
        listing_data = scrape_listings(driver, cities)
    finally:
        driver.quit()

    return listing_data




In [5]:
listing_data = airdna_scraper(city_list)

Logging in . . .
Searching for listings . . .
Detected 25 listings . . .
Name: San Juan
Score: 100
Info: Coastal
Info: $42.9k
Info: Revenue
Info: Potential
Info: 62%
Info: Occupancy
Info: $208.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Kauai
Score: 99
Info: Coastal
Info: $128.2k
Info: Revenue
Info: Potential
Info: 72%
Info: Occupancy
Info: $506.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Las Vegas
Score: 99
Info: Mid-size City
Info: $62.7k
Info: Revenue
Info: Potential
Info: 56%
Info: Occupancy
Info: $316.8
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Chattanooga
Score: 99
Info: Mid-size City
Info: $39.7k
Info: Revenue
Info: Potential
Info: 55%
Info: Occupancy
Info: $218.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'm

Could not extract listing data: Message: The element with the reference f9d8a3a6-c249-4074-9ba0-b9ea55f16415 is stale; either its node document is not the active document, or it is no longer connected to the DOM; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:725:5
getKnownElement@chrome://remote/content/marionette/json.sys.mjs:401:11
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:259:20
cloneObject/result<@chrome://remote/content/marionette/json.sys.mjs:53:52
cloneObject@chrome://remote/content/marionette/json.sys.mjs:53:25
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:289:16
cloneObject@chrome://remote/content/

Could not extract listing data: Message: The element with the reference fbc9807c-055f-43bb-b8d1-5af01c8aa699 is stale; either its node document is not the active document, or it is no longer connected to the DOM; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:725:5
getKnownElement@chrome://remote/content/marionette/json.sys.mjs:401:11
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:259:20
cloneObject/result<@chrome://remote/content/marionette/json.sys.mjs:53:52
cloneObject@chrome://remote/content/marionette/json.sys.mjs:53:25
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:289:16
cloneObject@chrome://remote/content/

Detected 38 listings . . .
Name: San Juan
Score: 100
Info: Coastal
Info: $42.9k
Info: Revenue
Info: Potential
Info: 62%
Info: Occupancy
Info: $208.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Kauai
Score: 99
Info: Coastal
Info: $128.2k
Info: Revenue
Info: Potential
Info: 72%
Info: Occupancy
Info: $506.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Las Vegas
Score: 99
Info: Mid-size City
Info: $62.7k
Info: Revenue
Info: Potential
Info: 56%
Info: Occupancy
Info: $316.8
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Chattanooga
Score: 99
Info: Mid-size City
Info: $39.7k
Info: Revenue
Info: Potential
Info: 55%
Info: Occupancy
Info: $218.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Flagstaff
Sc

Detected 38 listings . . .
Name: San Juan
Score: 100
Info: Coastal
Info: $42.9k
Info: Revenue
Info: Potential
Info: 62%
Info: Occupancy
Info: $208.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Kauai
Score: 99
Info: Coastal
Info: $128.2k
Info: Revenue
Info: Potential
Info: 72%
Info: Occupancy
Info: $506.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Las Vegas
Score: 99
Info: Mid-size City
Info: $62.7k
Info: Revenue
Info: Potential
Info: 56%
Info: Occupancy
Info: $316.8
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Chattanooga
Score: 99
Info: Mid-size City
Info: $39.7k
Info: Revenue
Info: Potential
Info: 55%
Info: Occupancy
Info: $218.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Flagstaff
Sc

Detected 38 listings . . .
Name: San Juan
Score: 100
Info: Coastal
Info: $42.9k
Info: Revenue
Info: Potential
Info: 62%
Info: Occupancy
Info: $208.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Kauai
Score: 99
Info: Coastal
Info: $128.2k
Info: Revenue
Info: Potential
Info: 72%
Info: Occupancy
Info: $506.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Las Vegas
Score: 99
Info: Mid-size City
Info: $62.7k
Info: Revenue
Info: Potential
Info: 56%
Info: Occupancy
Info: $316.8
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Chattanooga
Score: 99
Info: Mid-size City
Info: $39.7k
Info: Revenue
Info: Potential
Info: 55%
Info: Occupancy
Info: $218.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Flagstaff
Sc

Could not extract listing data: Message: The element with the reference add97324-d174-42b4-a20d-755b715c7857 is stale; either its node document is not the active document, or it is no longer connected to the DOM; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:725:5
getKnownElement@chrome://remote/content/marionette/json.sys.mjs:401:11
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:259:20
cloneObject/result<@chrome://remote/content/marionette/json.sys.mjs:53:52
cloneObject@chrome://remote/content/marionette/json.sys.mjs:53:25
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:289:16
cloneObject@chrome://remote/content/

Detected 38 listings . . .
Name: San Juan
Score: 100
Info: Coastal
Info: $42.9k
Info: Revenue
Info: Potential
Info: 62%
Info: Occupancy
Info: $208.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Kauai
Score: 99
Info: Coastal
Info: $128.2k
Info: Revenue
Info: Potential
Info: 72%
Info: Occupancy
Info: $506.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Las Vegas
Score: 99
Info: Mid-size City
Info: $62.7k
Info: Revenue
Info: Potential
Info: 56%
Info: Occupancy
Info: $316.8
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Chattanooga
Score: 99
Info: Mid-size City
Info: $39.7k
Info: Revenue
Info: Potential
Info: 55%
Info: Occupancy
Info: $218.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Flagstaff
Sc

Detected 38 listings . . .
Name: San Juan
Score: 100
Info: Coastal
Info: $42.9k
Info: Revenue
Info: Potential
Info: 62%
Info: Occupancy
Info: $208.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Kauai
Score: 99
Info: Coastal
Info: $128.2k
Info: Revenue
Info: Potential
Info: 72%
Info: Occupancy
Info: $506.1
Info: Daily Rate
Info: Over the last 12 months
Could not extract listing data: name 'max_listings' is not defined
Name: Las Vegas
Score: 99
Info: Mid-size City


KeyboardInterrupt: 